In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import warnings 
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import log_loss

# 통신데이터 합치기 및 카테고리별로 데이터프레임 나누기

In [2]:
for m in tqdm(range(1,10),desc='월단위 진행'):
    globals()["ts{}".format(m)] = pd.read_csv(f'./data/2022.{m}월_29개 통신정보.csv',encoding='cp949')
    try : 
        globals()["ts{}".format(m)].drop(columns=['행정동코드','총인구수','1인가구수'],inplace=True)    
    except:
        globals()["ts{}".format(m)].drop(columns=['행정동코드','총인구','1인가구수'],inplace=True)  
    globals()["ts{}".format(m)].drop(columns=globals()["ts{}".format(m)].filter(like='4분위수'),inplace=True) 
    # 데이터가 평균값을 제시하고 그 외 4분위 관련 수치를 따로 제공해주고 있음 -> 이미 전처리가 되어있는 거 같음..?
    globals()["ts{}".format(m)]['연령대_1'] = globals()["ts{}".format(m)]['연령대']//10*10
    globals()["ts{}".format(m)]['월'] = int(m)
    fix = globals()["ts{}".format(m)][['월','연령대_1','자치구','성별']]
    uni = ['상주지','출근','근무','소액결재','연체','카카오톡','SNS','통화','문자','데이터','이동','체류','지하철','게임','금융','쇼핑','동영상','유튜브','넷플릭스','배달']

    for col in tqdm(uni,desc=f'{m}월'):
        globals()["{}_{}".format(col,m)] = pd.concat([fix,globals()["ts{}".format(m)].filter(like=col)],axis=1)

for col in tqdm(uni,desc='카테고리별로 합치기'):
    globals()["{}".format(col)] = pd.DataFrame()
    for m in range(1,10):
        globals()["{}".format(col)]= pd.concat([globals()["{}".format(col)],globals()["{}_{}".format(col,m)]])


월단위 진행:   0%|          | 0/9 [00:00<?, ?it/s]

1월:   0%|          | 0/20 [00:00<?, ?it/s]

2월:   0%|          | 0/20 [00:00<?, ?it/s]

3월:   0%|          | 0/20 [00:00<?, ?it/s]

4월:   0%|          | 0/20 [00:00<?, ?it/s]

...

6월:   0%|          | 0/20 [00:00<?, ?it/s]

7월:   0%|          | 0/20 [00:00<?, ?it/s]

8월:   0%|          | 0/20 [00:00<?, ?it/s]

9월:   0%|          | 0/20 [00:00<?, ?it/s]

카테고리별로 합치기:   0%|          | 0/20 [00:00<?, ?it/s]

In [3]:
len(uni)

20

In [39]:
def lst_chunk(lst,n):
    return [lst[i:i+n] for i in range(0,len(lst),n)]

uni_dv = lst_chunk(uni,5)
uni_dv

[['상주지', '출근', '근무', '소액결재', '연체'],
 ['카카오톡', 'SNS', '통화', '문자', '데이터'],
 ['이동', '체류', '지하철', '게임', '금융'],
 ['쇼핑', '동영상', '유튜브', '넷플릭스', '배달']]

In [25]:
게임_8.shape[0], ts8.shape[0]

(10176, 10176)

In [3]:
연체.iloc[:,4:].describe()

,최근 3개월 내 요금 연체 비율
count,91584.000000
mean,11.044359
std,6.791881
min,0.000000
25%,5.580000
50%,9.020000
75%,16.400000
max,66.670000


# 카테고리별로 분류 모델링 돌리고 평가지표 확인(전처리 아무것도 안한 데이터)

In [4]:

import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance

In [3]:
uni_df = [상주지,출근,근무,소액결재,연체,카카오톡,SNS,통화,문자,데이터,이동,체류,지하철,게임,금융,쇼핑,동영상,유튜브,넷플릭스,배달]
uni = ['상주지','출근','근무','소액결재','연체','카카오톡','SNS','통화','문자','데이터','이동','체류','지하철','게임','금융','쇼핑','동영상','유튜브','넷플릭스','배달']

for name_df,name in zip(tqdm(uni_df),uni):
    test = name_df.drop(columns=['자치구','월'])
    y = test[['연령대_1']]
    X=test.drop(columns='연령대_1')
    
    #연령 인코딩 해주기
    le = LabelEncoder()
    result = le.fit_transform(y['연령대_1'])
    # print(le.classes_)
    
    # 성별 인코딩 해주기 
    le = LabelEncoder()
    X['성별'] = le.fit_transform(X['성별'])
    # print(le.classes_)

    X_train,X_test, y_train,y_test = train_test_split(X,result,test_size=0.2,random_state=11)
#     dt = DecisionTreeClassifier()
#     dt.fit(X_train,y_train)
#     pred_dt = dt.predict(X_test)
#     pred_proba_dt = dt.predict_proba(X_test)
    
#     rf = RandomForestClassifier()
#     rf.fit(X_train,y_train)
#     pred_rf = rf.predict(X_test)
#     pred_proba_rf = rf.predict_proba(X_test)
    
    # xgb= XGBClassifier()
    # xgb.fit(X_train,y_train)
    # pred_xgb = xgb.predict(X_test)
    # pred_proba_xgb = xgb.predict_proba(X_test)
    
    
    # print(f'=== [{name}] 정보 분류 모형 성능 평가 ===')
    # print(f'DT accuracy : {accuracy_score(y_test,pred_dt)} \nDT f1(macro) : {f1_score(y_test,pred_dt,average="macro")} \nDT logloss : {log_loss(y_test,pred_proba_dt)}\n')
    # print(f'RF accuracy : {accuracy_score(y_test,pred_rf)} \nRF f1(macro) : {f1_score(y_test,pred_rf,average="macro")} \nRF logloss : {log_loss(y_test,pred_proba_rf)}\n')
    # print(f'XGB accuracy : {accuracy_score(y_test,pred_xgb)} \nRF f1(macro) : {f1_score(y_test,pred_xgb,average="macro")} \nRF logloss : {log_loss(y_test,pred_proba_xgb)}\n')
    
    
    # print(classification_report(y_test,pred_rf))

  0%|          | 0/20 [00:00<?, ?it/s]

# Optuna test

In [ ]:
test = 배달.drop(columns=['자치구','월'])
y = test[['연령대_1']]
X=test.drop(columns='연령대_1')

y = pd.get_dummies(test['연령대_1'])
y

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

import optuna
from sklearn.metrics import roc_auc_score

def tree_objective(trial,X_train,y_train):  
    from sklearn.model_selection import cross_val_score
    params = dict(
        criterion = trial.suggest_categorical("criterion",['gini']), # 'gini', 'entropy','log_loss'
        splitter = trial.suggest_categorical("splitter",['best']), # 'best', 'random'
        max_depth = trial.suggest_int("max_depth", 20, 30),
        min_samples_split = trial.suggest_int("min_samples_split", 20, 30),
        min_samples_leaf = trial.suggest_int("min_samples_leaf", 50, 100),
        min_weight_fraction_leaf = trial.suggest_categorical("min_weight_fraction_leaf", [0.02]),
        # max_features = trial.suggest_categorical("max_features", [None]),
        max_leaf_nodes = trial.suggest_int("max_leaf_nodes", 50, 100),
        class_weight = trial.suggest_categorical("class_weight",['balanced']), # 'balanced', None
        ccp_alpha = trial.suggest_categorical("ccp_alpha", [0.0012])
    )     
    classifier_obj = DecisionTreeClassifier(**params, random_state=0)

    score = cross_val_score(classifier_obj, X_train, y_train, cv=5, scoring='roc_auc')
    roc_auc = score.mean()
    return roc_auc

# Optuna는 Study 개체를 기반으로 한다. 이 개체에는 필요한 파라미터 공간에 대한 정보와 sampler 방법과 pruning에 대한 모든 정보가 포함되어 있다.

tree_study = optuna.create_study(sampler=optuna.samplers.TPESampler(seed=100), direction="maximize")
tree_study.optimize(lambda trial :tree_objective(trial,X_train,y_train), n_trials=100)

model = DecisionTreeClassifier(**tree_study.best_params, random_state=0)
model.fit(X_train, y_train)
ROC_AUC = roc_auc_score(y_test, model.predict_proba(X_test)[0][:,1].reshape(-1,1))
print('ROC_AUC_score :', ROC_AUC)

In [40]:
pred_proba = model.predict_proba(X_test)

In [34]:
pred = model.predict(X_test)
pred

array([[0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0]], dtype=uint8)

In [41]:
pred_proba

[array([[0.91434423, 0.08565577],
        [0.99749489, 0.00250511],
        [0.89871966, 0.10128034],
        ...,
        [0.99802865, 0.00197135],
        [1.        , 0.        ],
        [0.39385854, 0.60614146]]),
 array([[0.35945322, 0.64054678],
        [0.99765215, 0.00234785],
        [0.96669395, 0.03330605],
        ...,
        [0.99690307, 0.00309693],
        [1.        , 0.        ],
        [0.68729709, 0.31270291]]),
 array([[0.8066026 , 0.1933974 ],
        [0.99719172, 0.00280828],
        [0.73076398, 0.26923602],
        ...,
        [0.63490601, 0.36509399],
        [0.98417021, 0.01582979],
        [0.95643605, 0.04356395]]),
 array([[0.99845571, 0.00154429],
        [0.91123714, 0.08876286],
        [0.72994079, 0.27005921],
        ...,
        [0.44847884, 0.55152116],
        [0.54874451, 0.45125549],
        [0.99697219, 0.00302781]]),
 array([[0.98950716, 0.01049284],
        [0.48282961, 0.51717039],
        [0.81417196, 0.18582804],
        ...,
        [

In [55]:
pred_proba[1][:,1].reshape(-1,1)

array([[0.64054678],
       [0.00234785],
       [0.03330605],
       ...,
       [0.00309693],
       [0.        ],
       [0.31270291]])